## Importación de librerías 

En esta sección se importan las principales librerías utilizadas en el preprocesamiento del texto.

- **spaCy**: biblioteca de procesamiento del lenguaje natural (NLP) que permite realizar tareas como tokenización, eliminación de stopwords o lematización.
    - Se incluye además la configuración personalizada del tokenizador de spaCy, que permite ajustar cómo se separan los tokens (por ejemplo, evitar divisiones innecesarias en palabras con guiones, puntos, etc.).
- **pandas**: biblioteca para la manipulación de datos estructurados (tablas) de tipo DataFrame.

In [1]:
# Librería que contiene herramientas de preprocesamiento
import spacy

# Herramientas internas de spaCy para modificar el tokenizador por defecto
from spacy.tokenizer import Tokenizer
from spacy.util import compile_infix_regex

# Librería para carga de datos
import pandas as pd

## Definición de funciones y configuración de preprocesamiento

Este bloque incluye todo lo necesario para preparar el texto antes de su análisis. Se utiliza el modelo `en_core_web_lg` de spaCy y se personaliza el tokenizador para evitar divisiones en guiones.

### Componentes definidos:

- **Carga y personalización del modelo spaCy**: se modifica el tokenizador para no dividir palabras por guiones.
- **`tokenizar_texto_spacy(texto)`**: convierte una cadena de texto a minúsculas y la transforma en una lista de tokens.
- **`eliminar_stopwords_spacy(tokens)`**: elimina tokens irrelevantes, incluyendo stopwords, palabras no alfabéticas y una lista personalizada de palabras excluidas.
- **`lematizar_texto_spacy(tokens)`**: transforma cada token en su lema o raíz.
- **`aplicar_mapeo(serie_de_listas, mapeo)`**: aplica un diccionario de equivalencias léxicas para unificar términos similares.
- **`palabras_extra`**: lista manual de palabras irrelevantes específicas del contexto del corpus.
- **`mapeo`**: diccionario de normalización construido a partir de un archivo `.csv` externo.


In [2]:
# Cargar el modelo grande de inglés (incluye vectores de palabras)
nlp = spacy.load("en_core_web_lg")

# Eliminar los guiones como separadores internos para evitar fragmentación de tokens como "ai-based" en "ai" "based"
infixes = [x for x in nlp.Defaults.infixes if "-" not in x]
infix_re = compile_infix_regex(infixes)

# Reconfigurar el tokenizador con las nuevas reglas de infijos
nlp.tokenizer = Tokenizer(
    nlp.vocab,
    rules=nlp.Defaults.tokenizer_exceptions,
    prefix_search=nlp.tokenizer.prefix_search,
    suffix_search=nlp.tokenizer.suffix_search,
    infix_finditer=infix_re.finditer,
    token_match=nlp.tokenizer.token_match
)



# Función de tokenización básica con spaCy en la que también se pasa el texto en minúsculas
# La separación en tokens se realiza según las reglas del modelo de lenguaje cargado,
# teniendo en cuenta espacios, puntuación, signos de puntuación interna (como guiones, apóstrofes, etc.)
# y excepciones predefinidas. En este caso, se ha modificado el tokenizer para que no divida por guiones.
def tokenizar_texto_spacy(texto):
    doc = nlp(texto.lower())
    return list(doc)




# Lista personalizada de palabras a ignorar (además de las stopwords de spaCy)
palabras_extra = [
    "co", "digital", "alexa", "anytime", "cortana", "arduino", "bitcoin",
    "deepl", "digitally", "digitise", "dropbox", "duckduckgo", "e", "etc",
    "eurostat", "example", "facebook", "google", "miro", "t", "twitter",
    "tpm", "youtube", "vs", "whatsapp", "wikipedia", "x", "en", "se", "lego",
    "non", "python", "ros", "scratch", "siri", "-", "non-digital", "one-time",
    "bas", "digital-based", "digitise", "digitised", "digitization", "s", "whilst"
]

# Asegurar que "3d" y "3-d" no se consideren stopwords, ya que el eliminador de stopwrods 
# elimina todas con números
nlp.vocab["3d"].is_alpha = True
nlp.vocab["3-d"].is_alpha = True
nlp.vocab["2d"].is_alpha = True
nlp.vocab["2-d"].is_alpha = True

# Función para eliminar stopwords y tokens no alfabéticos.
# Ignora las palabras_extra
def eliminar_stopwords_spacy(tokens):
    return [
        token for token in tokens
        if not token.is_stop
        and token.is_alpha
        and token.text.lower() not in palabras_extra
    ]
    
    
    
    
# Función de lematización del modelos cargado
def lematizar_texto_spacy(tokens):
    return [token.lemma_ for token in tokens]




# Función de mapeo de palabras en función de un diccionario de mapeo para normalización de términos
def aplicar_mapeo(serie_de_listas, mapeo):
    return serie_de_listas.apply(lambda tokens: [mapeo.get(token, token) for token in tokens])



#carga del archivo que contiene los mapeos correspondientes y conversión a tipo DataFrame para ser tratado como tabla
ruta_mapeo = "../utils/mapping.csv"
df_mapeo = pd.read_csv(ruta_mapeo)

# Construcción del diccionario: original -> equivalente
mapeo = dict(zip(df_mapeo["original"], df_mapeo["equivalente"]))

## Estructuración de datos de DIMENSIÓN 2
### Proceso

- **Datos**: 3 documentos Excel (todos identificados con `ID_area` e `ID_competencia`).
  1. Tabla de competencias
  2. Tabla de ejemplos
  3. Tabla de ejemplos de uso

- **Texto analizado**:
  - Descripción de la competencia: aporta una definición general.
  - Ejemplos: ilustran situaciones concretas de aplicación.
  - Casos de uso: detallan contextos específicos, objetivos y escenarios.

- **Proceso**:
  1. Recorro cada competencia por `id_area` e `id_competencia`.
  2. Para cada competencia, uno el texto de:
     - Descripción de la competencia.
     - Ejemplos.
     - Casos de uso.
  3. Genero dos tipos de tabla:
     - **Por competencia**:
       - **Competencia**: identificador de la competencia.
       - **Texto**: concatenación de los tres tipos de texto anteriores.
     - **Por área**:
       - **Área**: número de área.
       - **Texto**: unión de todos los textos de las competencias que pertenecen a esa área.

- **Objetivo**: Generar representaciones textuales por competencia y por área que permitan identificar patrones léxicos y temáticos, comparar contenidos y facilitar un análisis más completo del marco.

**Ejemplo tabla por competencia**:

| Área          | Competencia          | Texto                                      |
|---------------|----------------------|--------------------------------------------|
| Área 1        | Competencia 1        | Descripción + Ejemplo + Caso de uso        |
| Área 1        | Competencia 2        | Descripción + Ejemplo + Caso de uso        |

**Ejemplo tabla por área**:

| Área          | Texto concatenado de todas sus competencias        |
|---------------|----------------------------------------------------|
| Área 1        | Texto competencia 1 + competencia 2 + ...          |
| Área 2        | Texto competencia 5 + competencia 6 + ...          |


In [3]:
# Carga del archivo con información de la tabla Competencias y conversión a tipo DataFrame
ruta_competencias = "../data/digcomp/digcomp_competencias.csv"
competencias = pd.read_csv(ruta_competencias, encoding="latin1")

# Eliminación de registros en español para mantener solo los valores en inglés
competencias = competencias[competencias['Idioma'] != 'Español']

competencias.head()

,ID_competencia,Nombre,Descripción,ID_area,Idioma
1,1,"Browsing, searching and filtering data, inform...","To articulate information needs , to search fo...",1,Inglés
3,1,Interacting through digital technologies,To interact through a variety of digital techn...,2,Inglés
5,1,Developing digital content,To create and edit digital content in differen...,3,Inglés
7,1,Protecting devices,"To protect devices and digital content, and to...",4,Inglés
9,1,Solving technical problems,To identify technical problems when operating ...,5,Inglés


In [4]:
# Carga del archivo con información de la tabla Ejemplos y conversión a tipo DataFrame
ruta_ejemplos = "../data/digcomp/digcomp_ejemplos.csv"
ejemplos = pd.read_csv(ruta_ejemplos, encoding="latin1")

# Eliminación de registros en español para mantener solo los valores en inglés
ejemplos = ejemplos[ejemplos['Idioma'] != 'Español']

ejemplos.head()

,ID_ejemplo,ID_area,ID_competencia,Idioma,Texto,Tipo
1,1,1,1,Inglés,Knows that some online content in search resul...,Knowledge
3,2,1,1,Inglés,Aware that online content that is available to...,Knowledge
5,3,1,1,Inglés,"Aware that search results, social media activi...",Knowledge
7,4,1,1,Inglés,"Aware that search engines, social media and co...",Knowledge
9,5,1,1,Inglés,Aware that AI algorithms work in ways that are...,Knowledge


In [5]:
# Carga del archivo con información de la tabla Ejemplos de uso y conversión a tipo DataFrame
ruta_ejemplosUso = "../data/digcomp/digcomp_ejemplosUso.csv"
ejemplosUso = pd.read_csv(ruta_ejemplosUso, encoding="latin1")

# Eliminación de registros en español para mantener solo los valores en inglés
ejemplosUso = ejemplosUso[ejemplosUso['Idioma'] != 'Español']

ejemplosUso.head()


,ID_ejemploUso,ID_competencia,ID_area,Idioma,Nivel,Escenario,Objetivo,Contexto,Texto
1,1,1,1,Inglés,1,Employment,job seeking process,With help from an employment adviser,"I can identify, from a list, those job portals..."
3,2,1,1,Inglés,1,Employment,job seeking process,With help from an employment adviser,I can also find these job portals in my smartp...
5,3,1,1,Inglés,1,Employment,job seeking process,With help from an employment adviser,From a list of generic keywords for job seekin...
7,4,1,1,Inglés,1,Learning,prepare group work with my classmates,With help from my teacher,"I can identify websites, blogs and digital dat..."
9,5,1,1,Inglés,1,Learning,prepare group work with my classmates,With help from my teacher,I can also identify literature on the report t...


Primero se agrupan en 21 juegos de textos por las competencias. Es decir, todo el texto de la competencia 1 se junta y así con todas ellas.

In [6]:
# Agrupa los textos de ejemplos por área y competencia, concatenando los textos en una única cadena
ejemplos_grouped_21 = ejemplos.groupby(['ID_area', 'ID_competencia'])['Texto'].agg(' '.join).reset_index()
ejemplosUso_grouped_21 = ejemplosUso.groupby(['ID_area', 'ID_competencia'])['Texto'].agg(' '.join).reset_index()

# Renombra las columnas de texto para diferenciar ejemplos y ejemplos de uso
ejemplos_grouped_21 = ejemplos_grouped_21.rename(columns={'Texto': 'Texto ejemplos'})
ejemplosUso_grouped_21 = ejemplosUso_grouped_21.rename(columns={'Texto': 'Texto ejemplos de uso'})

display(ejemplos_grouped_21)

,ID_area,ID_competencia,Texto ejemplos
0,1,1,Knows that some online content in search resul...
1,1,2,Aware that online environments contain all typ...
2,1,3,Aware that many applications on the internet a...
3,2,1,Knows that many communication services (e.g. i...
4,2,2,Aware that everything that one shares publicly...
5,2,3,Knows about different types of digital service...
6,2,4,Aware of the advantages of using digital tools...
7,2,5,Aware of the meaning of non-verbal messages (e...
8,2,6,Aware that digital identity refers to (1) the ...
9,3,1,Knows that digital content exists in a digital...


In [7]:
# Une las descripciones de competencias con los textos de ejemplos y ejemplos de uso, usando las claves ID_area e ID_competencia
dimension_2_competencia = competencias.merge(ejemplos_grouped_21, on=['ID_area', 'ID_competencia'], how='left')
dimension_2_competencia = dimension_2_competencia.merge(ejemplosUso_grouped_21, on=['ID_area', 'ID_competencia'], how='left')

# Reordena las columnas para facilitar la lectura
dimension_2_competencia = dimension_2_competencia[['ID_area', 'ID_competencia', 'Descripción', 'Texto ejemplos', 'Texto ejemplos de uso']]
dimension_2_competencia = dimension_2_competencia.sort_values(by=['ID_area', 'ID_competencia']).reset_index(drop=True)

display(dimension_2_competencia)

,ID_area,ID_competencia,Descripción,Texto ejemplos,Texto ejemplos de uso
0,1,1,"To articulate information needs , to search fo...",Knows that some online content in search resul...,"I can identify, from a list, those job portals..."
1,1,2,"To analyse, compare and critically evaluate th...",Aware that online environments contain all typ...,I can identify in a list of job portals and ap...
2,1,3,"To organise, store and retrieve data, informat...",Aware that many applications on the internet a...,I can identify how and where to organise and k...
3,2,1,To interact through a variety of digital techn...,Knows that many communication services (e.g. i...,I can interact with participants and other col...
4,2,2,"To share data, information and digital content...",Aware that everything that one shares publicly...,I can use my companys digital storage system ...
5,2,3,To participate in society through the use of p...,Knows about different types of digital service...,I can propose and use different media strategi...
6,2,4,To use digital tools and technologies for coll...,Aware of the advantages of using digital tools...,I can use the most appropriate digital tools a...
7,2,5,To be aware of behavioural norms and know-how ...,Aware of the meaning of non-verbal messages (e...,"While organising an event for my organisation,..."
8,2,6,"To create, and manage one or multiple digital ...",Aware that digital identity refers to (1) the ...,I can propose to my boss a new social media pr...
9,3,1,To create and edit digital content in differen...,Knows that digital content exists in a digital...,I can find out how to add new dialogues and im...


In [8]:
# Crea una nueva columna 'Texto' combinando la descripción, los ejemplos y los ejemplos de uso
dimension_2_competencia['Texto'] = (
    dimension_2_competencia['Descripción'].fillna('') + ' ' +
    dimension_2_competencia['Texto ejemplos'].fillna('') + ' ' +
    dimension_2_competencia['Texto ejemplos de uso'].fillna('')
)

# Elimina las columnas individuales que ya se han fusionado
dimension_2_competencia = dimension_2_competencia.drop(
    ['Descripción', 'Texto ejemplos', 'Texto ejemplos de uso'],
    axis=1
)

display(dimension_2_competencia)

,ID_area,ID_competencia,Texto
0,1,1,"To articulate information needs , to search fo..."
1,1,2,"To analyse, compare and critically evaluate th..."
2,1,3,"To organise, store and retrieve data, informat..."
3,2,1,To interact through a variety of digital techn...
4,2,2,"To share data, information and digital content..."
5,2,3,To participate in society through the use of p...
6,2,4,To use digital tools and technologies for coll...
7,2,5,To be aware of behavioural norms and know-how ...
8,2,6,"To create, and manage one or multiple digital ..."
9,3,1,To create and edit digital content in differen...


Ahora se agrupan en juegos de 5, es decir, por cada área

In [9]:
# Une las tres tablas agrupadas por área general (ID_area)
ejemplos_grouped_5 = ejemplos.groupby('ID_area')['Texto'].agg(' '.join).reset_index()
ejemplosUso_grouped_5 = ejemplosUso.groupby('ID_area')['Texto'].agg(' '.join).reset_index()

# Renombra las columnas de texto para diferenciar ejemplos y ejemplos de uso
ejemplos_grouped_5 = ejemplos_grouped_5.rename(columns={'Texto': 'Texto ejemplos'})
ejemplosUso_grouped_5 = ejemplosUso_grouped_5.rename(columns={'Texto': 'Texto ejemplos de uso'})

# Agrupa las descripciones también por ID_area
competencias_grouped_5 = competencias.groupby('ID_area')['Descripción'].agg(' '.join).reset_index()

display(competencias_grouped_5)

,ID_area,Descripción
0,1,"To articulate information needs , to search fo..."
1,2,To interact through a variety of digital techn...
2,3,To create and edit digital content in differen...
3,4,"To protect devices and digital content, and to..."
4,5,To identify technical problems when operating ...


In [10]:
# Unir las tablas usando 'id_area' como clave
dimension_2_areas = competencias_grouped_5.merge(ejemplos_grouped_5, on=['ID_area'], how='left')
dimension_2_areas = dimension_2_areas.merge(ejemplosUso_grouped_5, on=['ID_area'], how='left')

# Reordena columnas
dimension_2_areas = dimension_2_areas[['ID_area', 'Descripción', 'Texto ejemplos', 'Texto ejemplos de uso']]
dimension_2_areas = dimension_2_areas.sort_values(by=['ID_area']).reset_index(drop=True)

display(dimension_2_areas)

,ID_area,Descripción,Texto ejemplos,Texto ejemplos de uso
0,1,"To articulate information needs , to search fo...",Knows that some online content in search resul...,"I can identify, from a list, those job portals..."
1,2,To interact through a variety of digital techn...,Knows that many communication services (e.g. i...,I can interact with participants and other col...
2,3,To create and edit digital content in differen...,Knows that digital content exists in a digital...,I can find out how to add new dialogues and im...
3,4,"To protect devices and digital content, and to...",Knows that using different strong passwords fo...,I can protect the corporate Twitter account us...
4,5,To identify technical problems when operating ...,Knows the main functions of the most common di...,I can create an illustrated video which answer...


In [11]:
# Crea la columna 'Texto' combinando descripción, ejemplos y ejemplos de uso por área
dimension_2_areas['Texto'] = (
    dimension_2_areas['Descripción'].fillna('') + ' ' +
    dimension_2_areas['Texto ejemplos'].fillna('') + ' ' +
    dimension_2_areas['Texto ejemplos de uso'].fillna('')
)

# Elimina columnas intermedias ya fusionadas
dimension_2_areas = dimension_2_areas.drop(
    ['Descripción', 'Texto ejemplos', 'Texto ejemplos de uso'],
    axis=1
)

display(dimension_2_areas)

,ID_area,Texto
0,1,"To articulate information needs , to search fo..."
1,2,To interact through a variety of digital techn...
2,3,To create and edit digital content in differen...
3,4,"To protect devices and digital content, and to..."
4,5,To identify technical problems when operating ...


## Estructuración de datos de DIMENSIÓN 3
### Proceso

- **Datos**: 2 documentos Excel:
  1. Tabla **Nivel** (con campos como `id_nivel`, que sus valores van del 1 al 8, y `Tipo`, que puede ser: Básico, Intermedio, Avanzado o Especialmente Avanzado).
  2. Tabla **Nivel de competencia** (con `id_area`, `id_competencia`, `id_nivel` y el campo `Texto`).

- **Texto analizado**:
  - El campo `Texto` de la tabla Nivel de competencia, que describe qué implica alcanzar cada nivel de dominio dentro de una competencia.

- **Proceso**:
  1. Se recorre la tabla Nivel de competencia agrupando textos de tres formas:
     - **Por competencia**: se agrupan los textos por combinación de `id_area` e `id_competencia`.
     - **Por tipo de nivel**: se agrupan todos los textos según el campo `Tipo` de la tabla Nivel (relacionado vía `id_nivel`).
     - **Por subnivel**: se agrupan los textos por `id_nivel` (que representa subniveles del 1 al 8).

- **Objetivo**: Obtener representaciones textuales que permitan analizar cómo se expresa el desarrollo progresivo de cada competencia, tanto desde una perspectiva individual (por competencia) como desde el punto de vista del tipo de nivel y subnivel.

**Ejemplo tabla por competencia**:

| Área          | Competencia          | Texto acumulado de niveles                |
|---------------|----------------------|--------------------------------------------|
| Área 1        | Competencia 1        | Texto nivel 1 de cada competencia + Texto nivel 2 de cada competencia + ...        |
| Área 1        | Competencia 2        | Texto nivel 1 de cada competencia + Texto nivel 2 de cada competencia + ...        |

**Ejemplo tabla por tipo de nivel**:

| Tipo de nivel          | Texto                                                                                                 |
|------------------------|-------------------------------------------------------------------------------------------------------|
| Básico                 | Texto nivel 1 de todas las competencias + Texto nivel 2 de todas las competencias                     |
| Intermedio             | Texto nivel 3 de todas las competencias + Texto nivel 4 de todas las competencias                     |
| Avanzado               | Texto nivel 5 de todas las competencias + Texto nivel 6 de todas las competencias                     |
| Especialmente avanzado | Texto nivel 7 de todas las competencias + Texto nivel 8 de todas las competencias                     |

**Ejemplo tabla por subnivel**:

| Subnivel (id_nivel)  | Texto                                                  |
|----------------------|--------------------------------------------------------|
| 1                    | Texto nivel 1 de todas las competencias                |
| 2                    | Texto nivel 2 de todas las competencias                |
| ...                  | ...                                                    |
| 8                    | Texto nivel 8 de todas las competencias                |


In [12]:
# Carga del archivo con información de la tabla Nivel de competencia y conversión a tipo DataFrame 
ruta_nivelCompetencia = "../data/digcomp/digcomp_nivelCompetencia.csv"
nivelCompetencia = pd.read_csv(ruta_nivelCompetencia, encoding="latin1")

# Eliminación de registros en español para mantener solo los valores en inglés
nivelCompetencia = nivelCompetencia[nivelCompetencia['Idioma'] != 'Español']

nivelCompetencia.head()

,ID_area,ID_competencia,Idioma,ID_nivel,ID_nivelCompetencia,Texto,Verbo
27,1,1,Inglés,1,1,"identify my information needs, find data, info...",identify
28,1,1,Inglés,1,2,"find how to access these data, information and...",find
29,1,1,Inglés,1,3,identify simple personal search strategies.,identify
30,1,1,Inglés,2,1,identify my information needs.,identify
31,1,1,Inglés,2,2,"find data, information and content through a s...",find


In [13]:
# Carga del archivo con información de la tabla Nivel y conversión a tipo DataFrame 
ruta_nivel = "../data/digcomp/digcomp_nivel.csv"
nivel = pd.read_csv(ruta_nivel, encoding="latin1")

# Eliminación de registros en español para mantener solo los valores en inglés
nivel = nivel[nivel['Idioma'] != 'Español']

nivel.head()

,ID_nivel,Idioma,Tipo,Descripción
1,1,Inglés,Foundation,"At basic level and with guidance, I can:"
3,2,Inglés,Foundation,At basic level and with autonomy and appropria...
5,3,Inglés,Intermediate,On my own and solving straightforward problems...
7,4,Inglés,Intermediate,"Independently, according to my own needs, and ..."
9,5,Inglés,Advanced,"As well as guiding others, I can:"


Unir ambas tablas por `id_nivel`

In [14]:
# Unir ambas tablas por 'id_nivel'
dimension_3_unida = nivelCompetencia.merge(nivel, on='ID_nivel', how='left')

display(dimension_3_unida)

,ID_area,ID_competencia,Idioma_x,ID_nivel,ID_nivelCompetencia,Texto,Verbo,Idioma_y,Tipo,Descripción
0,1,1,Inglés,1,1,"identify my information needs, find data, info...",identify,Inglés,Foundation,"At basic level and with guidance, I can:"
1,1,1,Inglés,1,2,"find how to access these data, information and...",find,Inglés,Foundation,"At basic level and with guidance, I can:"
2,1,1,Inglés,1,3,identify simple personal search strategies.,identify,Inglés,Foundation,"At basic level and with guidance, I can:"
3,1,1,Inglés,2,1,identify my information needs.,identify,Inglés,Foundation,At basic level and with autonomy and appropria...
4,1,1,Inglés,2,2,"find data, information and content through a s...",find,Inglés,Foundation,At basic level and with autonomy and appropria...
...,...,...,...,...,...,...,...,...,...,...
360,5,4,Inglés,6,3,choose the most appropriate opportunities for ...,choose,Inglés,Advanced,"At advanced level, according to my own needs a..."
361,5,4,Inglés,7,1,create solutions to complex problems with limi...,create,Inglés,Highly specialised,"At highly specialised level, I can:"
362,5,4,Inglés,7,2,integrate my knowledge to contribute to profes...,"integrate, guide",Inglés,Highly specialised,"At highly specialised level, I can:"
363,5,4,Inglés,8,1,create solutions to solve complex problems wit...,create,Inglés,Highly specialised,"At the most advanced and specialised level, I ..."


In [15]:
# Agrupa los textos de ejemplos por área y competencia, concatenando los textos en una única cadena
dimension_3_competencia = dimension_3_unida.groupby(['ID_area', 'ID_competencia'])['Texto'].agg(' '.join).reset_index()
dimension_3_competencia.columns = ['ID_area', 'ID_competencia', 'Texto']

display(dimension_3_competencia)

,ID_area,ID_competencia,Texto
0,1,1,"identify my information needs, find data, info..."
1,1,2,detect the credibility and reliability of comm...
2,1,3,"identify how to organise, store and retrieve d..."
3,2,1,select simple digital technologies to interact...
4,2,2,recognise simple appropriate digital technolog...
5,2,3,identify simple digital services in order to p...
6,2,4,choose simple digital tools and technologies f...
7,2,5,differentiate simple behavioural norms and kno...
8,2,6,"identify a digital identity, describe simple w..."
9,3,1,identify ways to create and edit simple conten...


In [16]:
# Agrupación por tipo de nivel (Básico, Intermedio, etc.)
dimension_3_tipoNivel = dimension_3_unida.groupby(['Tipo'])['Texto'].agg(' '.join).reset_index()
dimension_3_tipoNivel.columns = ['Tipo de nivel', 'Texto']

display(dimension_3_tipoNivel)

,Tipo de nivel,Texto
0,Advanced,respond to information needs. apply searches t...
1,Foundation,"identify my information needs, find data, info..."
2,Highly specialised,create solutions to complex problems with limi...
3,Intermediate,explain my information needs. perform well-def...


In [17]:
# Agrupación por subnivel (id_nivel)
dimension_3_subnivel = dimension_3_unida.groupby(['ID_nivel'])['Texto'].agg(' '.join).reset_index()
dimension_3_subnivel.columns = ['Subnivel', 'Texto']

display(dimension_3_subnivel)

,Subnivel,Texto
0,1,"identify my information needs, find data, info..."
1,2,"identify my information needs. find data, info..."
2,3,explain my information needs. perform well-def...
3,4,illustrate information needs. organise the sea...
4,5,respond to information needs. apply searches t...
5,6,assess information needs. adapt my searching s...
6,7,create solutions to complex problems with limi...
7,8,create solutions to solve complex problems wit...


## Estructuración de datos de DIMENSIÓN 4
### Proceso

- **Datos**: 1 documento Excel con la tabla **Ejemplos**, que contiene al menos los siguientes campos:
  - `id_area`
  - `id_competencia`
  - `Tipo` (puede ser: Conocimiento, Habilidad o Actitud)
  - `Texto` (contenido del ejemplo)

- **Texto analizado**:
  - El campo `Texto`, que recoge ejemplos relacionados con el desarrollo de cada competencia, clasificados en los tres tipos mencionados.

- **Proceso**:
  1. Se agrupan los textos de ejemplo de dos formas:
     - **Por competencia**: se agrupan todos los textos por combinación de `id_area` e `id_competencia`.
     - **Por tipo de ejemplo**: se agrupan los textos según el campo `Tipo`, que puede ser:
       - *Conocimiento*: comprensión teórica o conceptual.
       - *Habilidad*: acciones concretas realizadas por una persona.
       - *Actitud*: comportamientos, valores o disposiciones personales.

- **Objetivo**: Obtener representaciones textuales que permitan observar cómo se expresa el desarrollo de las competencias desde diferentes dimensiones: una individual (por competencia) y otra conceptual (por tipo de ejemplo), para facilitar un análisis comparativo y más completo.

**Ejemplo tabla por competencia**:

| Área          | Competencia          | Texto acumulado de ejemplos                          |
|---------------|----------------------|-------------------------------------------------------|
| Área 1        | Competencia 1        | Ejemplo 1 + Ejemplo 2 + Ejemplo 3                     |
| Área 1        | Competencia 2        | Ejemplo 1 + Ejemplo 2 + Ejemplo 3                     |

**Ejemplo tabla por tipo de ejemplo**:

| Tipo de ejemplo  | Texto acumulado                                                                   |
|------------------|------------------------------------------------------------------------------------|
| Conocimiento      | Ejemplo tipo conocimiento 1 + Ejemplo tipo conocimiento 2 + ...                   |
| Habilidad         | Ejemplo tipo habilidad 1 + Ejemplo tipo habilidad 2 + ...                         |
| Actitud           | Ejemplo tipo actitud 1 + Ejemplo tipo actitud 2 + ...                             |

**Ejemplo tabla por tipo de ejemplo**:

| Agrupación              | Texto acumulado                              |
|-------------------------|----------------------------------------------|
| Valor 1 de agrupación   | Texto acumulado de valor 1 de agrupación     |
| Valor 2 de agrupación   | Texto acumulado de valor 2 de agrupación     |
| Valor 3 de agrupación   | Texto acumulado de valor 3 de agrupación     |

No hace falta importar el dataset de ejemplos porque ya se importó para la dimensión 2

In [18]:
# Agrupación por competencia (id_area + id_competencia)
dimension_4_competencia = ejemplos.groupby(['ID_area', 'ID_competencia'])['Texto'].agg(' '.join).reset_index()
dimension_4_competencia.columns = ['ID_area', 'ID_competencia', 'Texto']

display(dimension_4_competencia)

,ID_area,ID_competencia,Texto
0,1,1,Knows that some online content in search resul...
1,1,2,Aware that online environments contain all typ...
2,1,3,Aware that many applications on the internet a...
3,2,1,Knows that many communication services (e.g. i...
4,2,2,Aware that everything that one shares publicly...
5,2,3,Knows about different types of digital service...
6,2,4,Aware of the advantages of using digital tools...
7,2,5,Aware of the meaning of non-verbal messages (e...
8,2,6,Aware that digital identity refers to (1) the ...
9,3,1,Knows that digital content exists in a digital...


In [19]:
# Agrupación por tipo de ejemplo (Conocimiento, Habilidad, Actitud)
dimension_4_tipoEjemplo = ejemplos.groupby('Tipo')['Texto'].agg(' '.join).reset_index()
dimension_4_tipoEjemplo.columns = ['Tipo', 'Texto']

display(dimension_4_tipoEjemplo)

,Tipo,Texto
0,Attitudes,Intentionally avoids distractions and aims to ...
1,Knowledge,Knows that some online content in search resul...
2,Skills,Can choose the search engine that most likely ...


## Obtención de lemas para la revisión de palabras

El análisis de frecuencias es una técnica fundamental en el preprocesamiento de texto que permite observar qué términos aparecen con mayor recurrencia en un corpus. En este caso, se utiliza para revisar el resultado del procesamiento y facilitar la toma de decisiones sobre dos aspectos clave:

- **Revisión de términos para mapeo léxico:** detectar palabras similares que podrían unificarse mediante reglas de mapeo.
- **Identificación de palabras irrelevantes:** encontrar términos frecuentes pero poco informativos, susceptibles de ser añadidos a una lista personalizada de stopwords.

### Proceso general

   - Preprocesamiento completo del texto (incluido stopwords con palabras adcicionales y mapeos consegudiso hasta el momento)
   - Agrupación de todos los tokens del corpus en el que se obtienen las frecuencias de cada palabra (con función `obtener_frecuencias`).
   - Exportación a archivo externo en el que se pueden analizar manualmente las palabras.

### Utilidad práctica

Este análisis permite:
- Detectar errores o inconsistencias en el preprocesamiento.
- Afinar los criterios de mapeo léxico para mejorar la representación semántica.
- Ampliar la lista de stopwords personalizadas con términos que resulten frecuentes pero irrelevantes en el dominio analizado.

In [20]:
# Paso 1: Unir y concatenar los textos de todas las tablas
textos_concatenados = pd.concat([
    competencias['Descripción'],
    nivel['Descripción'],
    nivelCompetencia['Texto'],
    ejemplos['Texto'],
    ejemplosUso['Texto']
], ignore_index=True)

# Paso 2: Aplicar el preprocesamiento completo a cada texto
textos_tokenizados = textos_concatenados.apply(tokenizar_texto_spacy)
textos_filtrados = textos_tokenizados.apply(eliminar_stopwords_spacy)
textos_lematizados = textos_filtrados.apply(lematizar_texto_spacy)
textos_mapeados = aplicar_mapeo(textos_lematizados, mapeo)

# Paso 3: Crear un DataFrame con los textos mapeados para explotar los tokens
tokens = pd.DataFrame({'Texto': textos_mapeados})

# Paso 4: Definición de función para obtener frecuencias
def obtener_frecuencias(df):
    # Explota la lista de palabras de la columna "Texto"
    df_explotado = df.explode("Texto")

    # Renombra la columna para mayor claridad
    df_explotado = df_explotado.rename(columns={"Texto": "Palabra"})

    # Agrupa por palabra y cuenta ocurrencias
    tabla_frecuencias = (
        df_explotado
        .groupby("Palabra")
        .size()
        .reset_index(name="Frecuencia")
        .sort_values(by="Frecuencia", ascending=False)
    )

    return tabla_frecuencias

# Paso 5: Calcular frecuencias
frecuencias = obtener_frecuencias(tokens)

# Paso 6: Exportar a CSV para revisión manual
frecuencias.to_csv("../utils/tokens_revisar_digcomp.csv", index=False)

## Aplicación del preprocesamiento a los textos de las dimensiones

En este bloque se aplica el flujo de preprocesamiento definido anteriormente (tokenización, eliminación de stopwords, lematización y mapeo) a las distintas tablas de texto correspondientes a varias dimensiones del marco de competencias.

Se procesan los siguientes conjuntos de datos:

- **Dimensión 2**
  - Áreas
  - Competencias

- **Dimensión 3**
  - Competencias
  - Tipos de nivel
  - Subniveles

- **Dimensión 4**
  - Competencias
  - Tipos de ejemplo

En cada caso, se transforma la columna `"Texto"` de cada tabla siguiendo el mismo flujo, con el objetivo de normalizar los textos y prepararlos para análisis posteriores (como comparación semántica o agrupamiento).


In [21]:
# Dimensión 2 - Áreas
dimension_2_areas_tokens = dimension_2_areas["Texto"].apply(tokenizar_texto_spacy)
dimension_2_areas_sinStopwords = dimension_2_areas_tokens.apply(eliminar_stopwords_spacy)
dimension_2_areas_lematizadas = dimension_2_areas_sinStopwords.apply(lematizar_texto_spacy)
dimension_2_areas["Texto"] = aplicar_mapeo(dimension_2_areas_lematizadas, mapeo)

# Dimensión 2 - Competencias
dimension_2_competencia_tokens = dimension_2_competencia["Texto"].apply(tokenizar_texto_spacy)
dimension_2_competencia_sinStopwords = dimension_2_competencia_tokens.apply(eliminar_stopwords_spacy)
dimension_2_competencia_lematizadas = dimension_2_competencia_sinStopwords.apply(lematizar_texto_spacy)
dimension_2_competencia["Texto"] = aplicar_mapeo(dimension_2_competencia_lematizadas, mapeo)

# Dimensión 3 - Competencias
dimension_3_competencia_tokens = dimension_3_competencia["Texto"].apply(tokenizar_texto_spacy)
dimension_3_competencia_sinStopwords = dimension_3_competencia_tokens.apply(eliminar_stopwords_spacy)
dimension_3_competencia_lematizadas = dimension_3_competencia_sinStopwords.apply(lematizar_texto_spacy)
dimension_3_competencia["Texto"] = aplicar_mapeo(dimension_3_competencia_lematizadas, mapeo)

# Dimensión 3 - Tipos de nivel
dimension_3_tipoNivel_tokens = dimension_3_tipoNivel["Texto"].apply(tokenizar_texto_spacy)
dimension_3_tipoNivel_sinStopwords = dimension_3_tipoNivel_tokens.apply(eliminar_stopwords_spacy)
dimension_3_tipoNivel_lematizadas = dimension_3_tipoNivel_sinStopwords.apply(lematizar_texto_spacy)
dimension_3_tipoNivel["Texto"] = aplicar_mapeo(dimension_3_tipoNivel_lematizadas, mapeo)

# Dimensión 3 - Subniveles
dimension_3_subnivel_tokens = dimension_3_subnivel["Texto"].apply(tokenizar_texto_spacy)
dimension_3_subnivel_sinStopwords = dimension_3_subnivel_tokens.apply(eliminar_stopwords_spacy)
dimension_3_subnivel_lematizadas = dimension_3_subnivel_sinStopwords.apply(lematizar_texto_spacy)
dimension_3_subnivel["Texto"] = aplicar_mapeo(dimension_3_subnivel_lematizadas, mapeo)

# Dimensión 4 - Competencias
dimension_4_competencia_tokens = dimension_4_competencia["Texto"].apply(tokenizar_texto_spacy)
dimension_4_competencia_sinStopwords = dimension_4_competencia_tokens.apply(eliminar_stopwords_spacy)
dimension_4_competencia_lematizadas = dimension_4_competencia_sinStopwords.apply(lematizar_texto_spacy)
dimension_4_competencia["Texto"] = aplicar_mapeo(dimension_4_competencia_lematizadas, mapeo)

# Dimensión 4 - Tipos de ejemplo
dimension_4_tipoEjemplo_tokens = dimension_4_tipoEjemplo["Texto"].apply(tokenizar_texto_spacy)
dimension_4_tipoEjemplo_sinStopwords = dimension_4_tipoEjemplo_tokens.apply(eliminar_stopwords_spacy)
dimension_4_tipoEjemplo_lematizadas = dimension_4_tipoEjemplo_sinStopwords.apply(lematizar_texto_spacy)
dimension_4_tipoEjemplo["Texto"] = aplicar_mapeo(dimension_4_tipoEjemplo_lematizadas, mapeo)

## Funciones para análisis de frecuencias por grupo

Este bloque permite calcular, filtrar y revisar frecuencias de palabras dentro de grupos definidos (por área, competencia, etc.).

### Componentes definidos:

- **`obtener_frecuencias_por_grupo(df, columnas_grupo)`**: genera una tabla de frecuencias por palabra dentro de cada grupo definido.
- **`filtrar_por_umbral_por_grupo(tabla, columnas_grupo)`**: filtra palabras frecuentes aplicando un umbral (media + desviación) dentro de cada grupo.
- **`imprimir_estadisticas_por_grupo(nombre, tabla_original, tabla_filtrada, columnas_grupo)`**: imprime estadísticas básicas del filtrado para cada grupo.


In [22]:
# Función para generar la tabla de frecuencias por grupo
def obtener_frecuencias_por_grupo(df, columnas_grupo):
    # Explota la columna "Texto", que contiene listas de palabras, en filas individuales
    df_explotado = df.explode("Texto")

    # Renombra la columna para que se entienda que cada fila representa una palabra
    df_explotado = df_explotado.rename(columns={"Texto": "Palabra"})

    # Agrupa por las columnas indicadas (grupo) y por palabra, y cuenta ocurrencias
    tabla_frecuencias = (
        df_explotado
        .groupby(columnas_grupo + ["Palabra"])
        .size()
        .reset_index(name="Frecuencia")
    )

    return tabla_frecuencias
    
    
# Función para filtrar palabras por grupo según un umbral
def filtrar_por_umbral_por_grupo(tabla_frecuencias, columnas_grupo):
    
    # Función interna que calcula el umbral y filtra un solo grupo
    def filtrar_grupo(grupo):
        media = grupo["Frecuencia"].mean()
        desviacion = grupo["Frecuencia"].std()
        umbral = media + desviacion
        return grupo[grupo["Frecuencia"] > umbral]
    
    # Aplica el filtrado a cada grupo y devuelve el resultado combinado
    return tabla_frecuencias.groupby(columnas_grupo, group_keys=False).apply(filtrar_grupo).reset_index(drop=True)


# Función para imprimir estadísticas descriptivas por grupo
def imprimir_estadisticas_por_grupo(nombre_tabla, tabla_frecuencias, tabla_frecuencias_filtradas, columnas_grupo):

    print(f"=== {nombre_tabla} ===")
    
    # Recorre cada grupo para calcular sus estadísticas y mostrar comparativas
    for nombre_grupo, grupo in tabla_frecuencias.groupby(columnas_grupo):
        total_original = grupo.shape[0]
        media = grupo["Frecuencia"].mean()
        desviacion = grupo["Frecuencia"].std()
        umbral = media + desviacion
        
        # Crea una máscara lógica para identificar el grupo correspondiente en la tabla filtrada
        if isinstance(nombre_grupo, tuple):
            mascara = True
            for col, val in zip(columnas_grupo, nombre_grupo):
                mascara &= (tabla_frecuencias_filtradas[col] == val)
        else:
            mascara = (tabla_frecuencias_filtradas[columnas_grupo] == nombre_grupo)
            
        grupo_filtrado = tabla_frecuencias_filtradas[mascara]
        total_filtrado = grupo_filtrado.shape[0]

        # Imprime resultados por grupo
        nombre = ", ".join(str(val) for val in nombre_grupo) if isinstance(nombre_grupo, tuple) else str(nombre_grupo)
        print(f"--- Grupo: {nombre} ---")
        print(f"Total de palabras (antes del filtrado): {total_original}")
        print(f"Total de palabras (después del filtrado): {total_filtrado}")
        print(f"Media de frecuencia: {media:.2f}")
        print(f"Desviación típica: {desviacion:.2f}")
        print(f"Umbral aplicado (media + desviación): {umbral:.2f}")
        print()


## Cálculo de frecuencias por dimensión y grupo

Este bloque ejecuta el análisis de frecuencias por grupos definidos en distintas dimensiones del marco competencial. Para cada grupo:

1. Se calcula la frecuencia total de palabras (`*_sinUmbral`).
2. Se filtran las más relevantes usando un umbral estadístico (`*_filtrado`).
3. Se imprimen estadísticas básicas para revisión.

### Dimensiones y agrupaciones utilizadas:

- **Dimensión 2**
  - Por área (`ID_area`)
  - Por competencia (`ID_area`, `ID_competencia`)
- **Dimensión 3**
  - Por competencia (`ID_area`, `ID_competencia`)
  - Por tipo de nivel (`Tipo de nivel`)
  - Por subnivel (`Subnivel`)
- **Dimensión 4**
  - Por competencia (`ID_area`, `ID_competencia`)
  - Por tipo de ejemplo (`Tipo`)


In [23]:
# Dimensión 2 - Áreas
frecuencias_d2_areas_sinUmbral = obtener_frecuencias_por_grupo(dimension_2_areas, ["ID_area"])
frecuencias_d2_areas = filtrar_por_umbral_por_grupo(frecuencias_d2_areas_sinUmbral, ["ID_area"])
imprimir_estadisticas_por_grupo("Dimension 2 - Areas", frecuencias_d2_areas_sinUmbral, frecuencias_d2_areas, ["ID_area"])

# Dimensión 2 - Competencias
frecuencias_d2_competencia_sinUmbral = obtener_frecuencias_por_grupo(dimension_2_competencia, ["ID_area", "ID_competencia"])
frecuencias_d2_competencia = filtrar_por_umbral_por_grupo(frecuencias_d2_competencia_sinUmbral, ["ID_area", "ID_competencia"])
imprimir_estadisticas_por_grupo("Dimensión 2 - Competencias", frecuencias_d2_competencia_sinUmbral, frecuencias_d2_competencia, ["ID_area", "ID_competencia"])

# Dimensión 3 - Competencias
frecuencias_d3_competencia_sinUmbral = obtener_frecuencias_por_grupo(dimension_3_competencia, ["ID_area", "ID_competencia"])
frecuencias_d3_competencia = filtrar_por_umbral_por_grupo(frecuencias_d3_competencia_sinUmbral, ["ID_area", "ID_competencia"])
imprimir_estadisticas_por_grupo("Dimensión 3 - Competencias", frecuencias_d3_competencia_sinUmbral, frecuencias_d3_competencia, ["ID_area", "ID_competencia"])

# Dimensión 3 - Tipo de Nivel
frecuencias_d3_tipoNivel_sinUmbral = obtener_frecuencias_por_grupo(dimension_3_tipoNivel, ["Tipo de nivel"])
frecuencias_d3_tipoNivel = filtrar_por_umbral_por_grupo(frecuencias_d3_tipoNivel_sinUmbral, ["Tipo de nivel"])
imprimir_estadisticas_por_grupo("Dimensión 3 - Tipo de Nivel", frecuencias_d3_tipoNivel_sinUmbral, frecuencias_d3_tipoNivel, ["Tipo de nivel"])

# Dimensión 3 - Subnivel
frecuencias_d3_subnivel_sinUmbral = obtener_frecuencias_por_grupo(dimension_3_subnivel, ["Subnivel"])
frecuencias_d3_subnivel = filtrar_por_umbral_por_grupo(frecuencias_d3_subnivel_sinUmbral, ["Subnivel"])
imprimir_estadisticas_por_grupo("Dimensión 3 - Subnivel", frecuencias_d3_subnivel_sinUmbral, frecuencias_d3_subnivel, ["Subnivel"])

# Dimensión 4 - Competencias
frecuencias_d4_competencia_sinUmbral = obtener_frecuencias_por_grupo(dimension_4_competencia, ["ID_area", "ID_competencia"])
frecuencias_d4_competencia = filtrar_por_umbral_por_grupo(frecuencias_d4_competencia_sinUmbral, ["ID_area", "ID_competencia"])
imprimir_estadisticas_por_grupo("Dimensión 4 - Competencias", frecuencias_d4_competencia_sinUmbral, frecuencias_d4_competencia, ["ID_area", "ID_competencia"])

# Dimensión 4 - Tipo de Ejemplo
frecuencias_d4_tipoEjemplo_sinUmbral = obtener_frecuencias_por_grupo(dimension_4_tipoEjemplo, ["Tipo"])
frecuencias_d4_tipoEjemplo = filtrar_por_umbral_por_grupo(frecuencias_d4_tipoEjemplo_sinUmbral, ["Tipo"])
imprimir_estadisticas_por_grupo("Dimensión 4 - Tipo de Ejemplo", frecuencias_d4_tipoEjemplo_sinUmbral, frecuencias_d4_tipoEjemplo, ["Tipo"])

=== Dimension 2 - Areas ===
--- Grupo: 1 ---
Total de palabras (antes del filtrado): 372
Total de palabras (después del filtrado): 27
Media de frecuencia: 2.38
Desviación típica: 3.07
Umbral aplicado (media + desviación): 5.46

--- Grupo: 2 ---
Total de palabras (antes del filtrado): 578
Total de palabras (después del filtrado): 50
Media de frecuencia: 2.76
Desviación típica: 3.85
Umbral aplicado (media + desviación): 6.61

--- Grupo: 3 ---
Total de palabras (antes del filtrado): 356
Total de palabras (después del filtrado): 24
Media de frecuencia: 2.58
Desviación típica: 3.60
Umbral aplicado (media + desviación): 6.18

--- Grupo: 4 ---
Total de palabras (antes del filtrado): 466
Total de palabras (después del filtrado): 40
Media de frecuencia: 2.51
Desviación típica: 3.29
Umbral aplicado (media + desviación): 5.80

--- Grupo: 5 ---
Total de palabras (antes del filtrado): 391
Total de palabras (después del filtrado): 35
Media de frecuencia: 2.43
Desviación típica: 2.88
Umbral aplicado 

C:\Users\mario.fernandezr\AppData\Local\Temp\ipykernel_1388\3679962933.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return tabla_frecuencias.groupby(columnas_grupo, group_keys=False).apply(filtrar_grupo).reset_index(drop=True)
C:\Users\mario.fernandezr\AppData\Local\Temp\ipykernel_1388\3679962933.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return tabla_frecuencias.groupby(columnas_grupo, gr

## Construcción de matrices de frecuencias por grupo

En este bloque se preparan las matrices de frecuencias necesarias para los análisis posteriores. Para ello:

1. **Se crea una columna combinada `Comp`** que concatena `ID_area` e `ID_competencia` para los casos en los que las agrupaciones sean las competencias.
2. **Se generan matrices de frecuencias** mediante `pivot_table()`, en las que:
   - Las filas corresponden a palabras.
   - Las columnas representan los distintos grupos.
   - Los valores son las frecuencias de aparición de cada palabra en cada grupo.
3. **Se exportan las matrices a archivos `.csv`**.

Este paso transforma los datos textuales en estructuras matriciales.

### Ejemplo simplificado

**Antes del pivoteo:**

| Palabra   | Área             | Frecuencia |
|-----------|------------------|------------|
| access    | 1                | 3          |
| digital   | 1                | 2          |
| access    | 2                | 5          |

**Después del pivoteo:**

| Palabra   | Área 1    | Área 2   |
|-----------|-----------|----------|
| access    | 3         | 5        |
| digital   | 2         | 0        |





In [24]:
# Columna "Comp" que combina los dos valores `ID_area` e `ID_competencia`
frecuencias_d2_competencia["Comp"] = frecuencias_d2_competencia["ID_area"].astype(str) + "." + frecuencias_d2_competencia["ID_competencia"].astype(str)
frecuencias_d3_competencia["Comp"] = frecuencias_d3_competencia["ID_area"].astype(str) + "." + frecuencias_d3_competencia["ID_competencia"].astype(str)
frecuencias_d4_competencia["Comp"] = frecuencias_d4_competencia["ID_area"].astype(str) + "." + frecuencias_d4_competencia["ID_competencia"].astype(str)

# Pivotar la tabla para obtener una matriz
# filas = palabras, columnas = agrupaciones, valores = frecuencias
# Dimensión 2 - Áreas
matriz_frecuencias_d2_areas = frecuencias_d2_areas.pivot_table(index="Palabra", columns="ID_area", values="Frecuencia", fill_value=0)
matriz_frecuencias_d2_areas.columns.name = "Palabra"
matriz_frecuencias_d2_areas.index.name = None
matriz_frecuencias_d2_areas.to_csv("../results/digcomp/frecuencias_d2_areas.csv")

# Dimensión 2 - Competencias
matriz_frecuencias_d2_competencia = frecuencias_d2_competencia.pivot_table(index="Palabra", columns="Comp", values="Frecuencia", fill_value=0)
matriz_frecuencias_d2_competencia.columns.name = "Palabra"
matriz_frecuencias_d2_competencia.index.name = None
matriz_frecuencias_d2_competencia.to_csv("../results/digcomp/frecuencias_d2_competencia.csv")

# Dimensión 3 - Competencias
matriz_frecuencias_d3_competencia = frecuencias_d3_competencia.pivot_table(index="Palabra",columns="Comp",values="Frecuencia",fill_value=0)
matriz_frecuencias_d3_competencia.index.name = "Palabra"
matriz_frecuencias_d3_competencia.columns.name = None
matriz_frecuencias_d3_competencia.to_csv("../results/digcomp/frecuencias_d3_competencia.csv")

# Dimensión 3 - Tipo de Nivel
matriz_frecuencias_d3_tipoNivel = frecuencias_d3_tipoNivel.pivot_table(index="Palabra",columns="Tipo de nivel",values="Frecuencia",fill_value=0)
matriz_frecuencias_d3_tipoNivel.index.name = None
matriz_frecuencias_d3_tipoNivel.columns.name = "Palabra"
matriz_frecuencias_d3_tipoNivel.to_csv("../results/digcomp/frecuencias_d3_tipoNivel.csv")

# Dimensión 3 - Subnivel
matriz_frecuencias_d3_subnivel = frecuencias_d3_subnivel.pivot_table(index="Palabra",columns="Subnivel",values="Frecuencia",fill_value=0)
matriz_frecuencias_d3_subnivel.index.name = None
matriz_frecuencias_d3_subnivel.columns.name = "Palabra"
matriz_frecuencias_d3_subnivel.to_csv("../results/digcomp/frecuencias_d3_subnivel.csv")

# Dimensión 4 - Competencias
matriz_frecuencias_d4_competencia = frecuencias_d4_competencia.pivot_table(index="Palabra",columns="Comp",values="Frecuencia",fill_value=0)
matriz_frecuencias_d4_competencia.index.name = None
matriz_frecuencias_d4_competencia.columns.name = "Palabra"
matriz_frecuencias_d4_competencia.to_csv("../results/digcomp/frecuencias_d4_competencia.csv")

# Dimensión 4 - Tipo de ejemplo
matriz_frecuencias_d4_tipoEjemplo = frecuencias_d4_tipoEjemplo.pivot_table(index="Palabra",columns="Tipo",values="Frecuencia",fill_value=0)
matriz_frecuencias_d4_tipoEjemplo.index.name = None
matriz_frecuencias_d4_tipoEjemplo.columns.name = "Palabra"
matriz_frecuencias_d4_tipoEjemplo.to_csv("../results/digcomp/frecuencias_d4_tipoEjemplo.csv")
